In [3]:
import ee
import pandas as pd
import requests
from io import StringIO

# Initialize the Earth Engine module.
# ee.Authenticate()
ee.Initialize()

# URL of the CSV file
url = 'https://raw.githubusercontent.com/orwell2024/uscrnlib/main/extract_slides/2024stations_days.csv'

# Read the CSV file from the URL
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

# Limiting to first 10 locations for testing purposes
data = data.head(4)

# Function to create and export image for a given location
def create_and_export_image(row):
    station_name = row['Station']
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']
    
    point = ee.Geometry.Point([longitude, latitude])
    buffer = point.buffer(10000).bounds()  # 5000 meters buffer for 10 km x 10 km region
    
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterBounds(buffer) \
                    .filterDate('2023-04-11', '2023-6-01') \
                    .sort('CLOUDY_PIXEL_PERCENTAGE') \
                    .first()
    
    # Get the date of the image
    date = ee.Date(collection.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    
    # Select the relevant bands
    image = collection.select(['B4', 'B3', 'B2']).clip(buffer)
    
    # Export the image with smaller dimensions
    export_task = ee.batch.Export.image.toDrive(
        image=image,
        description=f"{station_name}_{date}",
        folder='GEE_Images',
        scale=10,
        region=buffer.getInfo()['coordinates'],
        fileFormat='GeoTIFF',  # Export as GeoTIFF
        maxPixels=1e8
    )
    
    export_task.start()

# Apply the function to each row in the dataframe
data.apply(create_and_export_image, axis=1)

print("Export tasks have been started.")


Export tasks have been started.


In [16]:
import ee
import pandas as pd
import requests
from io import StringIO

# Initialize the Earth Engine module.
# ee.Authenticate()
ee.Initialize()

# URL of the CSV file
url = 'https://raw.githubusercontent.com/orwell2024/uscrnlib/main/extract_slides/2024stations_days.csv'

# Read the CSV file from the URL
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

# Filter the data to include only stations from Alabama (AL)
data_al = data[data['Station'].str.startswith('TX_')].copy()

# Initialize lists to store results
built_1975_percent_list = []
built_2020_percent_list = []
percentage_change_list = []

# Function to create and export image for a given location
def process_location(row):
    station_name = row['Station']
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']
    sizeKm = 10  # Size of the cell in kilometers

    # Define a point for the center of the rectangle at the specified coordinates
    centerPoint = ee.Geometry.Point([longitude, latitude])

    # Create a bounding box around the center point
    halfSideLength = (sizeKm / 2) * 1000  # Convert km to meters
    cell = centerPoint.buffer(halfSideLength).bounds()

    # Load the built-up surface images for 1975 and 2020 from the JRC GHSL dataset
    image_1975 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/1975').select('built_surface')
    image_2020 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/2020').select('built_surface')

    # Clip the built-up images to the cell
    built_1975_clipped = image_1975.clip(cell)
    built_2020_clipped = image_2020.clip(cell)

    # Calculate the average built-up value for the cell in 1975
    mean1975 = built_1975_clipped.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=cell,
        scale=30,
        maxPixels=1e9
    ).get('built_surface').getInfo()

    # Calculate the average built-up value for the cell in 2020
    mean2020 = built_2020_clipped.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=cell,
        scale=30,
        maxPixels=1e9
    ).get('built_surface').getInfo()

    # Normalize to percentage of the area (1% = 10,000 square meters per hectare)
    percentage1975 = round((mean1975 / 10000) * 100, 2) if mean1975 is not None else 0
    percentage2020 = round((mean2020 / 10000) * 100, 2) if mean2020 is not None else 0

    # Calculate the percentage change
    percentage_change = round(((percentage2020 - percentage1975) / percentage1975) * 100, 2) if percentage1975 != 0 else None

    # Append results to lists
    built_1975_percent_list.append(percentage1975)
    built_2020_percent_list.append(percentage2020)
    percentage_change_list.append(percentage_change)

    # Export the 1975 image to Google Drive
    export_task_1975 = ee.batch.Export.image.toDrive(
        image=built_1975_clipped,
        description=f'Built_up_surface_1975_{station_name}_{sizeKm}km_cell',
        folder='GEE_Images',
        scale=30,
        region=cell,
        maxPixels=1e9
    )
    export_task_1975.start()

    # Export the 2020 image to Google Drive
    export_task_2020 = ee.batch.Export.image.toDrive(
        image=built_2020_clipped,
        description=f'Built_up_surface_2020_{station_name}_{sizeKm}km_cell',
        folder='GEE_Images',
        scale=30,
        region=cell,
        maxPixels=1e9
    )
    export_task_2020.start()

# Apply the function to each row in the filtered dataframe
data_al.apply(process_location, axis=1)

# Add the results to the dataframe
data_al.loc[:, 'Built_1975_percent'] = built_1975_percent_list
data_al.loc[:, 'Built_2020_percent'] = built_2020_percent_list
data_al.loc[:, 'Percentage_Change'] = percentage_change_list

# Save the updated dataframe to a new CSV file
output_csv_path = 'updated_stations_data_al.csv'
data_al.to_csv(output_csv_path, index=False)

print("Export tasks have been started and results have been written to the CSV file.")


Export tasks have been started and results have been written to the CSV file.


In [31]:
import ee
import pandas as pd
import requests
from io import StringIO

# Initialize the Earth Engine module.
# ee.Authenticate()
ee.Initialize()

# URL of the CSV file
url = 'https://raw.githubusercontent.com/orwell2024/uscrnlib/main/extract_slides/2024stations_days.csv'

# Read the CSV file from the URL
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

# Filter the data to include only stations from Texas (TX)
data_tx = data#[data['Station'].str.startswith('TX_')].copy()

# Initialize lists to store results
results = {
    "Built_1975_50km_percent": [],
    "Built_2020_50km_percent": [],
    "Percentage_Change_50km": [],
    "Built_1975_10km_percent": [],
    "Built_2020_10km_percent": [],
    "Percentage_Change_10km": [],
    "Built_1975_2km_percent": [],
    "Built_2020_2km_percent": [],
    "Percentage_Change_2km": []
}

# Function to process location and calculate built-up surface percentages
def process_location(row, sizeKm):
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']

    # Define a point for the center of the rectangle at the specified coordinates
    centerPoint = ee.Geometry.Point([longitude, latitude])

    # Create a bounding box around the center point
    halfSideLength = (sizeKm / 2) * 1000  # Convert km to meters
    cell = centerPoint.buffer(halfSideLength).bounds()

    # Load the built-up surface images for 1975 and 2020 from the JRC GHSL dataset
    image_1975 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/1975').select('built_surface')
    image_2020 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/2020').select('built_surface')

    # Clip the built-up images to the cell
    built_1975_clipped = image_1975.clip(cell)
    built_2020_clipped = image_2020.clip(cell)

    # Calculate the average built-up value for the cell in 1975
    mean1975 = built_1975_clipped.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=cell,
        scale=30,
        maxPixels=1e9
    ).get('built_surface').getInfo()

    # Calculate the average built-up value for the cell in 2020
    mean2020 = built_2020_clipped.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=cell,
        scale=30,
        maxPixels=1e9
    ).get('built_surface').getInfo()

    # Normalize to percentage of the area (1% = 10,000 square meters per hectare)
    percentage1975 = round((mean1975 / 10000) * 100, 2) if mean1975 is not None else 0
    percentage2020 = round((mean2020 / 10000) * 100, 2) if mean2020 is not None else 0

    # Calculate the percentage change
    if percentage1975 == 0:
        percentage_change = 0
    else:
        percentage_change = round(((percentage2020 - percentage1975) / percentage1975) * 100, 2)

    return percentage1975, percentage2020, percentage_change

# Process each location for different cell sizes
for index, row in data_tx.iterrows():
    # Process for 50km cell
    result_50km = process_location(row, 50)
    results["Built_1975_50km_percent"].append(result_50km[0])
    results["Built_2020_50km_percent"].append(result_50km[1])
    results["Percentage_Change_50km"].append(result_50km[2])
    
    # Process for 10km cell
    result_10km = process_location(row, 10)
    results["Built_1975_10km_percent"].append(result_10km[0])
    results["Built_2020_10km_percent"].append(result_10km[1])
    results["Percentage_Change_10km"].append(result_10km[2])
    
    # Process for 2km cell
    result_2km = process_location(row, 2)
    results["Built_1975_2km_percent"].append(result_2km[0])
    results["Built_2020_2km_percent"].append(result_2km[1])
    results["Percentage_Change_2km"].append(result_2km[2])

# Add the results to the dataframe
for key, value in results.items():
    data_tx[key] = value

# Save the updated dataframe to a new CSV file
output_csv_path = 'updated_stations_data_tx.csv'
data_tx.to_csv(output_csv_path, index=False)

print("Processing is complete and results have been written to the CSV file.")


Processing is complete and results have been written to the CSV file.


In [32]:
import ee
import pandas as pd
import requests
from io import StringIO
import re
import time, random
from datetime import datetime

# Initialize the Earth Engine module.
# ee.Authenticate()
ee.Initialize()

# URL of the data
url = 'https://data.giss.nasa.gov/gistemp/station_data_v4_globe/v4.temperature.inv.txt'

# Fetch the data from the URL
response = requests.get(url)
data_text = response.text

# Clean up the data format
data_text = data_text.replace("Station Name", "Station")
data_text = re.sub(r"[ \t]+", ";", data_text)

while ';;' in data_text:
    data_text = data_text.replace(";;", ";")

data_lines = data_text.split('\n')
data_lines = [line.rstrip(';') for line in data_lines]

cleaned_data_text = "\n".join(data_lines)

if not cleaned_data_text.startswith("ID;Lat;Lon;Elev-m;Station;BI"):
    cleaned_data_text = cleaned_data_text.replace("D;Lat;Lon;Elev-m;Station;BI", "ID;Lat;Lon;Elev-m;Station;BI", 1)

data_io = StringIO(cleaned_data_text)

valid_rows = []

for line in data_io:
    fields = line.split(';')
    if len(fields) == 6:
        valid_rows.append(fields)

data = pd.DataFrame(valid_rows, columns=["ID", "Lat", "Lon", "Elev-m", "Station", "BI"])

data['Lat'] = pd.to_numeric(data['Lat'], errors='coerce')
data['Lon'] = pd.to_numeric(data['Lon'], errors='coerce')
data['BI'] = pd.to_numeric(data['BI'], errors='coerce')

data = data.dropna(subset=['Lat', 'Lon', 'BI'])

# Initialize lists to store results
results = {
    "ID": [],
    "Station": [],
    "BI": [],
    "Built_1975_50km_percent": [],
    "Built_2020_50km_percent": [],
    "Percentage_Change_50km": [],
    "Built_1975_10km_percent": [],
    "Built_2020_10km_percent": [],
    "Percentage_Change_10km": [],
    "Built_1975_2km_percent": [],
    "Built_2020_2km_percent": [],
    "Percentage_Change_2km": []
}

# Function to process location and calculate built-up surface percentages
def process_location(row, sizeKm):
    latitude = row['Lat']
    longitude = row['Lon']

    # Define a point for the center of the rectangle at the specified coordinates
    centerPoint = ee.Geometry.Point([longitude, latitude])

    # Create a bounding box around the center point
    halfSideLength = (sizeKm / 2) * 1000  # Convert km to meters
    cell = centerPoint.buffer(halfSideLength).bounds()

    # Load the built-up surface images for 1975 and 2020 from the JRC GHSL dataset
    image_1975 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/1975').select('built_surface')
    image_2020 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/2020').select('built_surface')

    # Clip the built-up images to the cell
    built_1975_clipped = image_1975.clip(cell)
    built_2020_clipped = image_2020.clip(cell)

    # Calculate the average built-up value for the cell in 1975
    mean1975 = built_1975_clipped.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=cell,
        scale=30,
        maxPixels=1e9
    ).get('built_surface').getInfo()

    # Calculate the average built-up value for the cell in 2020
    mean2020 = built_2020_clipped.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=cell,
        scale=30,
        maxPixels=1e9
    ).get('built_surface').getInfo()

    # Normalize to percentage of the area (1% = 10,000 square meters per hectare)
    percentage1975 = round((mean1975 / 10000) * 100, 2) if mean1975 is not None else 0
    percentage2020 = round((mean2020 / 10000) * 100, 2) if mean2020 is not None else 0

    # Calculate the percentage change
    if percentage1975 == 0:
        percentage_change = 0
    else:
        percentage_change = round(((percentage2020 - percentage1975) / percentage1975) * 100, 2)

    return percentage1975, percentage2020, percentage_change

# Function to process a batch of locations
def process_batch(batch_data):
    for index, row in batch_data.iterrows():
        # Process for 50km cell
        result_50km = process_location(row, 50)
        results["ID"].append(row["ID"])
        results["Station"].append(row["Station"])
        results["BI"].append(row["BI"])
        results["Built_1975_50km_percent"].append(result_50km[0])
        results["Built_2020_50km_percent"].append(result_50km[1])
        results["Percentage_Change_50km"].append(result_50km[2])
        
        # Process for 10km cell
        result_10km = process_location(row, 10)
        results["Built_1975_10km_percent"].append(result_10km[0])
        results["Built_2020_10km_percent"].append(result_10km[1])
        results["Percentage_Change_10km"].append(result_10km[2])
        
        # Process for 2km cell
        result_2km = process_location(row, 2)
        results["Built_1975_2km_percent"].append(result_2km[0])
        results["Built_2020_2km_percent"].append(result_2km[1])
        results["Percentage_Change_2km"].append(result_2km[2])

    # Create a DataFrame for the batch results
    batch_df = pd.DataFrame(results)
    
    # Save the batch results to a CSV file
    batch_df.to_csv('updated_stations_data_tx.csv', mode='a', index=False, header=not pd.io.common.file_exists('updated_stations_data_tx.csv'))

    # Clear the results for the next batch
    for key in results.keys():
        results[key].clear()

# Process the data in batches of 1000
batch_size = 100
print(len(data))
for start_index in range(2000, len(data), batch_size):
    print ("starting batch  ", start_index, "  ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    end_index = min(start_index + batch_size, len(data))
    batch_data = data.iloc[start_index:end_index]
    process_batch(batch_data)
    #sleep_time = random.randint(20, 100)
    #time.sleep(sleep_time)  # Wait for 1 minute before processing the next batch

print("Processing is complete and results have been written to the CSV file.")


27519
starting batch   2000    2024-07-03 11:16:28
starting batch   2100    2024-07-03 11:18:45
starting batch   2200    2024-07-03 11:20:38
starting batch   2300    2024-07-03 11:23:03
starting batch   2400    2024-07-03 11:26:03
starting batch   2500    2024-07-03 11:29:20
starting batch   2600    2024-07-03 11:33:30
starting batch   2700    2024-07-03 11:36:57
starting batch   2800    2024-07-03 11:40:09
starting batch   2900    2024-07-03 11:43:11
starting batch   3000    2024-07-03 11:46:10
starting batch   3100    2024-07-03 11:48:57
starting batch   3200    2024-07-03 11:51:47
starting batch   3300    2024-07-03 11:54:54
starting batch   3400    2024-07-03 11:58:04
starting batch   3500    2024-07-03 12:01:09
starting batch   3600    2024-07-03 12:03:59
starting batch   3700    2024-07-03 12:07:02
starting batch   3800    2024-07-03 12:09:54
starting batch   3900    2024-07-03 12:12:52
starting batch   4000    2024-07-03 12:15:37
starting batch   4100    2024-07-03 12:18:42
star

In [44]:
import pandas as pd
import requests
from io import StringIO
import re

# URL of the data
url = 'https://data.giss.nasa.gov/gistemp/station_data_v4_globe/v4.temperature.inv.txt'

# Fetch the data from the URL
response = requests.get(url)
data_text = response.text

# Clean up the data format
data_text = data_text.replace("Station Name", "Station")
data_text = re.sub(r"[ \t]+", ";", data_text)

while ';;' in data_text:
    data_text = data_text.replace(";;", ";")

data_lines = data_text.split('\n')
data_lines = [line.rstrip(';') for line in data_lines]

cleaned_data_text = "\n".join(data_lines)

#if not cleaned_data_text.startswith("ID;Lat;Lon;Elev-m;Station;BI"):
#    cleaned_data_text = cleaned_data_text.replace("D;Lat;Lon;Elev-m;Station;BI", "ID;Lat;Lon;Elev-m;Station;BI", 1)

data_io = StringIO(cleaned_data_text)

valid_rows = []

for line in data_io:
    fields = line.split(';')
    if len(fields) == 6:
        valid_rows.append(fields)

data_original = pd.DataFrame(valid_rows, columns=["ID", "Lat", "Lon", "Elev-m", "Station", "BI"])

# Convert the numeric columns from strings to appropriate numeric types
data_original['Lat'] = pd.to_numeric(data_original['Lat'], errors='coerce')
data_original['Lon'] = pd.to_numeric(data_original['Lon'], errors='coerce')
data_original['Elev-m'] = pd.to_numeric(data_original['Elev-m'], errors='coerce')
data_original['BI'] = pd.to_numeric(data_original['BI'], errors='coerce')

# Load the existing CSV file
output_csv_path = 'updated_stations_data_tx.csv'
data_existing = pd.read_csv(output_csv_path)

# Debug: Print columns before merging
print("Columns in original data:", data_original.columns)
print("Columns in existing data:", data_existing.columns)

# Ensure 'ID' column is present in both DataFrames
if 'ID' not in data_existing.columns:
    print("'ID' column not found in existing data. Please check the input file.")
else:
    # Merge the original data with the existing data on the "ID" column
    data_merged = pd.merge(data_existing, data_original[["ID", "Lat", "Lon", "Elev-m", "BI"]], on="ID", how="left")

    # Rename columns to avoid confusion
    data_merged = data_merged.rename(columns={'BI_x': 'BI', 'BI_y': 'Original_BI'})

    # Save the updated DataFrame back to a CSV file
    data_merged.to_csv('updated_stations_data_tx_all.csv', index=False)

    print("Columns added and CSV file updated.")

# Read the result file and check columns
data_reordered = pd.read_csv('updated_stations_data_tx_all.csv')

# Debug: Print columns after merging and saving
print("Columns in the merged data:", data_reordered.columns)

# Reorder columns if they all exist
columns_order = [
    "ID", "Station", "Lat", "Lon", "Elev-m", "BI",
    "Built_1975_50km_percent", "Built_2020_50km_percent", "Percentage_Change_50km",
    "Built_1975_10km_percent", "Built_2020_10km_percent", "Percentage_Change_10km",
    "Built_1975_2km_percent", "Built_2020_2km_percent", "Percentage_Change_2km"
]

missing_columns = [col for col in columns_order if col not in data_reordered.columns]
if missing_columns:
    print("Missing columns in the merged data:", missing_columns)
else:
    data_reordered = data_reordered[columns_order]
    data_reordered.to_csv('reordered_stations_data_tx_all.csv', index=False)
    print("Columns reordered and CSV file saved.")


Columns in original data: Index(['ID', 'Lat', 'Lon', 'Elev-m', 'Station', 'BI'], dtype='object')
Columns in existing data: Index(['ID', 'Station', 'BI', 'Built_1975_50km_percent',
       'Built_2020_50km_percent', 'Percentage_Change_50km',
       'Built_1975_10km_percent', 'Built_2020_10km_percent',
       'Percentage_Change_10km', 'Built_1975_2km_percent',
       'Built_2020_2km_percent', 'Percentage_Change_2km'],
      dtype='object')
Columns added and CSV file updated.
Columns in the merged data: Index(['ID', 'Station', 'BI', 'Built_1975_50km_percent',
       'Built_2020_50km_percent', 'Percentage_Change_50km',
       'Built_1975_10km_percent', 'Built_2020_10km_percent',
       'Percentage_Change_10km', 'Built_1975_2km_percent',
       'Built_2020_2km_percent', 'Percentage_Change_2km', 'Lat', 'Lon',
       'Elev-m', 'Original_BI'],
      dtype='object')
Columns reordered and CSV file saved.


In [38]:
import pandas as pd

# Load the merged CSV file
input_csv_path = 'updated_stations_data_tx_all.csv'
data_merged = pd.read_csv(input_csv_path)

# Define the desired column order
columns_order = [
    "ID", "Station", "Lat", "Lon", "Elev-m", "BI",
    "Built_1975_50km_percent", "Built_2020_50km_percent", "Percentage_Change_50km",
    "Built_1975_10km_percent", "Built_2020_10km_percent", "Percentage_Change_10km",
    "Built_1975_2km_percent", "Built_2020_2km_percent", "Percentage_Change_2km"
]

# Reorder the columns
data_merged = data_merged[columns_order]

# Save the reordered DataFrame to a new CSV file
output_csv_path = 'reordered_stations_data_tx_all.csv'
data_merged.to_csv(output_csv_path, index=False)

print("Columns reordered and CSV file saved.")


Columns reordered and CSV file saved.


In [65]:
import pandas as pd
from geopy.distance import geodesic

# Function to check if two locations are approximately equal
def is_approx_equal(lat1, lon1, lat2, lon2, tol=0.01):
    return abs(lat1 - lat2) < tol and abs(lon1 - lon2) < tol

# Function to calculate the distance between two points in kilometers
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Load the previously created CSV file
previous_csv_path = 'reordered_stations_data_tx_all.csv'
previous_data = pd.read_csv(previous_csv_path)

# Load the new CSV file from GitHub
new_csv_url = 'https://raw.githubusercontent.com/orwell2024/uscrnlib/main/extract_slides/2024stations_days.csv'
new_data = pd.read_csv(new_csv_url)

# Add the USCRN_Y_N column with default value 'N'
previous_data['USCRN_Y_N'] = 'N'

# Ensure the USCRN_Station column exists and is of type object (string)
previous_data['USCRN_Station'] = ""

# List to keep track of matched and missing stations
matched_stations = []
missing_stations = []

# Iterate over the new data and check for matches in the previous data
i = 0
for index, row in new_data.iterrows():
    station_name = row['Station']
    lat = row['LATITUDE']
    lon = row['LONGITUDE']
    
    # Check for approximate match in the previous data
    match = previous_data.apply(lambda x: is_approx_equal(x['Lat'], x['Lon'], lat, lon), axis=1)
    
    if match.any():
        i += 1
        matched_index = match.idxmax()
        original_station_name = previous_data.loc[matched_index, 'Station']
        matched_stations.append((original_station_name, station_name, previous_data.loc[matched_index, 'ID']))
        # Update the USCRN_Y_N column to 'Y' where there is a match
        previous_data.loc[matched_index, 'USCRN_Y_N'] = 'Y'
        # Add the new station name
        previous_data.loc[matched_index, 'USCRN_Station'] = station_name
    else:
        missing_stations.append((station_name, lat, lon))

# Print the matched stations count and missing stations
print(f"Total matched stations: {i}")
print("Matched USCRN stations:")
for original_station, uscrn_station, station_id in matched_stations:
    print(f"Original Station: {original_station}, USCRN Station: {uscrn_station}, ID: {station_id}")

print("Missing USCRN stations:")
for station_name, lat, lon in missing_stations:
    print(station_name)

# Find the closest GHCN location for each missing USCRN station
print("\nClosest GHCN locations for missing USCRN stations:")
for station_name, uscrn_lat, uscrn_lon in missing_stations:
    min_distance = float('inf')
    closest_ghcn_station = None
    closest_ghcn_lat = None
    closest_ghcn_lon = None
    
    for index, row in previous_data.iterrows():
        ghcn_lat = row['Lat']
        ghcn_lon = row['Lon']
        distance = calculate_distance((uscrn_lat, uscrn_lon), (ghcn_lat, ghcn_lon))
        
        if distance < min_distance:
            min_distance = distance
            closest_ghcn_station = row['Station']
            closest_ghcn_lat = ghcn_lat
            closest_ghcn_lon = ghcn_lon
            
    print(f"USCRN Station: {station_name}, Closest GHCN Station: {closest_ghcn_station}, Distance: {min_distance:.2f} km")

# Reorder columns to make USCRN_Y_N the third column
cols = list(previous_data.columns)
cols.insert(2, cols.pop(cols.index('USCRN_Y_N')))
previous_data = previous_data[cols]

# Save the reordered DataFrame back to the CSV file
updated_csv_path = 'updated_stations_data_with_uscrn.csv'
previous_data.to_csv(updated_csv_path, index=False)

print("Updated CSV file with USCRN_Y_N column saved.")


Total matched stations: 139
Matched USCRN stations:
Original Station: FAIRBANKS_11_NE, USCRN Station: AK_Fairbanks_11_NE, ID: USW00026494
Original Station: PAXSON, USCRN Station: AK_Glennallen_64_N, ID: USC00507097
Original Station: KENAI_29_ENE, USCRN Station: AK_Kenai_29_ENE, ID: USW00026563
Original Station: METLAKATLA_6_S, USCRN Station: AK_Metlakatla_6_S, ID: USW00025381
Original Station: PORT_ALSWORTH, USCRN Station: AK_Port_Alsworth_1_SW, ID: USC00507570
Original Station: SAND_POINT_1_ENE, USCRN Station: AK_Sand_Point_1_ENE, ID: USW00025630
Original Station: SITKA_1_NE, USCRN Station: AK_Sitka_1_NE, ID: USW00025379
Original Station: YAKUTAT_STATE_AP, USCRN Station: AK_Yakutat_3_SSE, ID: USW00025339
Original Station: BREWTON_3_ENE, USCRN Station: AL_Brewton_3_NNE, ID: USC00011080
Original Station: CLANTON_2_NE, USCRN Station: AL_Clanton_2_NE, ID: USW00063891
Original Station: COURTLAND_2_WSW, USCRN Station: AL_Courtland_2_WSW, ID: USW00063868
Original Station: CULLMAN_NAHS, USCRN